In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import muon_functions as mf

## Pulling from the Neutron Output File:

The Neutron Output file `NEWINPUT001_fort.99` contains lines in the following format:

` Neutron in TPC! NCASE, ETRACK, LTRACK, WTRACK`<br>
`       6       1.3184171765016091          5       1.0000000000000000`

The goal here is to read in the files, creating the following dataframe with each line being a neutron:

|Muon|Energy|Generation|Muon Impact Parameter|
|----|----------------------|------|----------|
|6|1.3184171765016091|0.0|5|

## Muon format in phase space file:
|Fluka  Num.|Energy [GeV]|$X_0$|$Y_0$|$Z_0$|$\cos x$|$\cos y$|$\cos z$|Stat. Weight|
|----------------|-------------|----|------|----|---------|--------|--------|------------|
|11| 1111.5767| -20.3335| -9.14201| 22.2000| 0.6735| 0.1693| -0.71946| 1|

In [11]:
import pandas as pd
import numpy as np
import muon_functions as mf

def read_neutron_file_and_muons(neutron_path, phase_space) -> pd.DataFrame:

    # Loading up a pandas dataframe 

    muons, energy, generation = [], [], []
    xsco, ysco, zsco = [], [], []
    cx, cy, cz = [], [], []
    with open(neutron_path) as neutron_file:
        line = neutron_file.readline()
        while line:
            lin_array = line.split()

            muons.append(int(lin_array[0]))
            energy.append(float(lin_array[1]))
            generation.append(int(lin_array[2]))
            xsco.append(float(lin_array[3]))
            ysco.append(float(lin_array[4]))
            zsco.append(float(lin_array[5]))
            cx.append(float(lin_array[6]))
            cy.append(float(lin_array[7]))
            cz.append(float(lin_array[8]))
            
            line = neutron_file.readline()

    # Pass the data from the lists into the dataframes
    neutron_data = pd.DataFrame(columns = ['muon', 'energy', 'generation', 'xsco', 'ysco', 'zsco', 'cx', 'cy', 'cz'])

    neutron_data['muon'], neutron_data['energy'], neutron_data['generation'] = muons, energy, generation
    neutron_data['xsco'], neutron_data['ysco'], neutron_data['zsco'] = xsco, ysco, zsco
    neutron_data['cx'], neutron_data['cy'], neutron_data['cz'] = cx, cy, cz

    del muons, energy, generation, xsco, ysco, zsco, cx, cy, cz # Delete the unneeded lists

    muon_list = neutron_data['muon'].unique()

    muon_data = pd.DataFrame(columns = ['energy', 'impact', 'x0', 'y0', 'z0', 'zenith', 'azimuth'])

    # Loading in the muon data from the phase space file
    muons = []
    with open(phase_space) as muon_file:
        line = muon_file.readline()
        line_number = 1 # The line we're on in the file ; the muon number according to the file: [1, e.o.f]
        muon_number = 0 # The number of the muon we're interested in taking: muon_list[some_index]
        while line:
            if line_number == muon_list[muon_number]:
                # Append the muon to the list
                lin_array = line.split()
                muons.append(lin_array)
                muon_number = muon_number + 1
                if muon_number == len(muon_list): break # ladies and gentlemen, we got 'em
            line_number = line_number + 1
            line = muon_file.readline()

    muon_index = 0
    impact_parameters = []
    for muon in muon_list:

        temp = muons[muon_index]
        zenith = np.arccos(float(temp[7]))
        energy = float(temp[1])
        azimuth = np.arccos(float(temp[5])/np.sin(zenith))
        initial =  (float(temp[2]), float(temp[3]), float(temp[4]))
        tempMuon = mf.Muon(zenith, azimuth, 0, initial)
        impact_param = tempMuon.closest_approach((0,0,0))

        new_muon = {'energy' : [energy], 'impact' : [impact_param], 'zenith': zenith, 'azimuth': azimuth,\
             'x0' : [initial[0]], 'y0' : [initial[1]], 'z0' : [initial[1]]}
        new_muon_df = pd.DataFrame(new_muon)
        muon_data = pd.concat([muon_data, new_muon_df], ignore_index=True)

    #     # set 'impact' = impact_param for every entry in neutron_data that has muon as its muon
    #     for index, row in neutron_data.iterrows():

    #         if int(row['muon']) == int(muon_list[muon_index]):
    #             impact_parameters.append(impact_param)

    #     muon_index += 1
        
    # neutron_data['impact'] = impact_parameters
    
    return neutron_data, muon_data


In [12]:
neutron_data = read_neutron_file_and_muons('NEWINPUT001_fort.99', 'muon_file.txt')
type(neutron_data)

tuple

# Plotting the data

Now we want to make a scatter plot of the number of neutrons produced per muon versus the impact parameter


In [8]:
impacts = neutron_data['impact'].unique()
neutron_count = []

for impact in impacts:
    neutron_count.append(len(neutron_data.loc[neutron_data['impact'] == impact]))
plt.scatter(impacts, neutron_count)

TypeError: tuple indices must be integers or slices, not str

In [8]:
neutron_data = read_neutron_file_and_muons(neutron_path='NEWINPUT001_fort.99', phase_space='muon_file.txt')
impacts = neutron_data['impact'].unique()
neutron_count = []

for impact in impacts:
    neutron_count.append(len(neutron_data.loc[neutron_data['impact'] == impact]))
plt.scatter(impacts, neutron_count)

FileNotFoundError: [Errno 2] No such file or directory: 'NEWINPUT001_fort.99'

In [7]:
neutron_data

,muon,energy,generation,impact
0,2448,0.939566,36,10.392439
1,2448,0.940821,7,10.392439
2,4690,1.142912,8,22.058195
